<a href="https://colab.research.google.com/github/Narusi/My-Projects/blob/master/RevolutPortfolioNewAssets%20v1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

In [0]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import pearsonr, spearmanr

import requests
from bs4 import BeautifulSoup

import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=3)

In [4]:
def momentumBuy(close, asset='Asset', step=50, output='current', chart=False, printOut=False):
    shortBuy = close > close.rolling(step).mean()
    mediumBuy = close > close.rolling(step * 2).mean()
    longBuy = close > close.rolling(step * 4).mean()

    medMBuy = close.rolling(step).mean() > close.rolling(step * 2).mean()
    longMBuy = close.rolling(step).mean() > close.rolling(step * 4).mean()

    veryLongBuy = close.rolling(step * 2).mean() > close.rolling(200).mean()

    buySignals = pd.DataFrame(index=close.index)

    buySignals['BuySS'] = shortBuy
    buySignals['BuySM'] = mediumBuy
    buySignals['BuySL'] = longBuy
    buySignals['BuyMM'] = medMBuy
    buySignals['BuyML'] = longMBuy
    buySignals['BuyLL'] = veryLongBuy

    ts = pd.DataFrame({asset:close.div(close.iloc[0]),
                    'Returns':np.log(close/close.shift(1)),
                    'BuySignal':buySignals.sum(axis=1)},
                    index=close.index)

    momSum = ts.groupby('BuySignal')['Returns'].sum()
    momMed = ts.groupby('BuySignal')['Returns'].median()
    buySigns = []
    holdSignal = []
    sellSign = []
    losingSignals = momSum[momSum <= 0.0].index

    for bl in range(len(buySignals.columns),0,-1):        
        if np.logical_and(momSum[bl] > 0.0, bl > max(losingSignals)):
            buySigns.append(bl)
        elif momSum[bl] > 0.0:
            holdSignal.append(bl)
        elif momMed[bl] <= 0.0:
            sellSign.append(bl)

    ts['Signal'] = ''
    ts.loc[ts[ts['BuySignal'] == 0].index, 'Signal'] = 'SELL'
    
    if len(buySigns) > 1:
        for buy in buySigns:
            ts.loc[ts[ts['BuySignal'] == buy].index, 'Signal'] = 'BUY'
    elif len(buySigns) == 1:
        ts.loc[ts[ts['BuySignal'] == buySigns[0]].index, 'Signal'] = 'BUY'
        
    if len(holdSignal) > 1:
        for hold in holdSignal:
            ts.loc[ts[ts['BuySignal'] == hold].index, 'Signal'] = 'HOLD'
    elif len(holdSignal) == 1:
        ts.loc[ts[ts['BuySignal'] == holdSignal[0]].index, 'Signal'] = 'HOLD'

    if len(sellSign) > 1:
        for sell in sellSign:
            ts.loc[ts[ts['BuySignal'] == sell].index, 'Signal'] = 'SELL'
    elif len(sellSign) == 1:
        ts.loc[ts[ts['BuySignal'] == sellSign[0]].index, 'Signal'] = 'SELL'

    for b in buySignals.columns:    
        rets = ts['Returns'][buySignals[b]]
        prob = rets[rets > 0.0].count()/rets.count()    
        if printOut: print(b, prob, rets.mean());

        ts[b] = 0
        ts.loc[buySignals[buySignals[b]].index, b] = 1

    if chart:
        ts['2018':][[asset,'BuySignal']].plot(figsize=(16,8))
        plt.show();

    if printOut: print(ts.groupby('BuySignal')['Returns'].mean());
    
    for i in range(6):
        minpick = ts.loc[ts[ts['BuySignal'] >= i].index]
        probs = minpick.groupby('BuySignal')['Returns'].count()/minpick['Returns'].count()    
        rets = minpick.groupby('BuySignal')['Returns'].mean()
        std = minpick.groupby('BuySignal')['Returns'].std()

        dprobs = minpick[minpick['Returns'] < 0.0].groupby('BuySignal')['Returns'].count()/minpick.groupby('BuySignal')['Returns'].count()
        #print((probs * rets).sum())
        #print(dprobs)
    
    ts['LAG.14'] = ts['Returns'].shift(14).rolling(14).sum()
    ts['LAG.28'] = ts['Returns'].shift(28).rolling(28).sum()
    ts['LAG.60'] = ts['Returns'].shift(60).rolling(60).sum()
    ts['LAG.90'] = ts['Returns'].shift(90).rolling(90).sum()
    lags = ['LAG.14','LAG.28','LAG.60','LAG.90']
    
    signalProbs = pd.DataFrame(index=list(range(len(list(itertools.product([1, 0], 
                                                                           repeat = len(buySignals.columns)))))),
                               columns=list(buySignals.columns) + lags)
    signalProbs[buySignals.columns] = 0
    signalProbs[lags] = 0.0

    signalReturns = pd.DataFrame(index=list(range(len(list(itertools.product([1, 0], 
                                                                           repeat = len(buySignals.columns)))))),
                               columns=list(buySignals.columns) + lags)
    signalReturns[buySignals.columns] = 0
    signalReturns[lags] = 0.0

    i = 0
    for comb in itertools.product([1, 0], repeat = len(buySignals.columns)):
        comb = list(comb)
        signalProbs.loc[i, buySignals.columns] = comb

        signInd = buySignals[buySignals == comb].dropna().index
        signRets = ts[lags].reindex(signInd)
        theProbs = signRets[signRets > 0.07/step].count()/signRets.count()
        #print(comb)
        #print(buySignals[buySignals == comb].dropna().tail())
        #print(theProbs)
        signalProbs.loc[i, lags] = theProbs
        signalReturns.loc[i, lags] = signRets.median()
        i += 1
    
    for ss in buySignals.columns:
        rets = ts[ts[ss] == 1][lags].dropna()
        prob = rets[rets > 0.01].count()/rets.count()
        hrets = rets[rets > 0.01].mean()

        if printOut:
            print('\n',ss)
            print(rets.mean(), '\n',rets.median())
            print('\nProbabilities:')
            print(prob.mean())
            print((prob * hrets).mean())
    
    if printOut: print(ts.groupby('Signal')['Returns'].mean());
    
    if output == 'current' or output == 'single':
        currentState = (np.array(signalProbs[buySignals.columns].values) == np.array(ts[buySignals.columns].iloc[-1].values)).sum(axis=1) == len(buySignals.columns)
        lagOutput = []
        for lagProb in lags:
            action = ''
            if signalProbs[currentState][lagProb].values[0] > 0.75: action = 'BUY'
            elif signalProbs[currentState][lagProb].values[0] > 0.50: action = 'HOLD'
            elif signalProbs[currentState][lagProb].values[0] <= 0.50: action = 'SELL'

            lagOutput.append([lagProb, action, 
                            signalProbs[currentState][lagProb].values[0], 
                            signalReturns[currentState][lagProb].values[0]])

            if printOut:
                print(lagProb, action, 
                    round(100*signalProbs[currentState][lagProb].values[0],1),
                    round(100*signalReturns[currentState][lagProb].values[0],3))                
        
        if output == 'single':
            ttemp = pd.DataFrame(lagOutput)        
            return ttemp.loc[ttemp[2].idxmin(), 1]
        else:
            return lagOutput

    elif output == 'ts':
        return ts

a = 'BABA'
pps = yf.download(a)['Adj Close']
#print(pps.tail())
timeSeries = momentumBuy(pps, asset=a, output='ts')
print('Current state {}\n\n'.format(timeSeries['BuySignal'].iloc[-1]), 
      timeSeries.groupby(['Signal','BuySignal'])['Returns'].mean())

print(pd.DataFrame(momentumBuy(pps, asset=a, output='current'),
                   columns=list(['LAG','Signal','Profit Prob.','Return'])))

momentumBuy(pps, asset=a, output='single')

[*********************100%***********************]  1 of 1 completed
Current state 6

 Signal  BuySignal
        4           -0.001974
BUY     5            0.002428
        6            0.003457
SELL    0           -0.001219
        1           -0.000397
        2           -0.001300
        3           -0.001956
Name: Returns, dtype: float64
      LAG Signal  Profit Prob.    Return
0  LAG.14    BUY      0.843990  0.039354
1  LAG.28    BUY      0.833760  0.074049
2  LAG.60    BUY      0.772379  0.107942
3  LAG.90   HOLD      0.647059  0.107781


'HOLD'

In [5]:
portfolio = ['T', 'PFE','FB','MSFT','AMTD']
candidates = ['AAPL','ABBV','ABT','ACN','ADI','AGM','AGNC','ALB','AMTD','AMZN',
              'APD','ARR','AVAL','BABA','BAH','BEN','BLK','CAH','CATY','CBT',
              'CCBG','CCL','CHD','CHRW','CFG','CL','CMA','CNI','CMCSA','COST',
              'CSCO','CVS','CVX','D','DDS','DUK','DX','EBF','EMR','ENB','EPD',
              'EPR','ES','FAST','FB','FDX','FITB','FRT','FLIC','FLO','GIS','GGG',
              'GOOG','GS','GWB','HBAN','IBCP','IBM','IMBBY','INTC','IRM','JNJ',
              'JPM','K','KEY','KMB','KO','LMT','LOW','MA','MAIN','MBWM','MCY','MCD',
              'MDP','MDT','MED','MNRL','MMM','MMP','MO','MPLX','MSFT','MSI','MSM','NHI','NFLX',
              'NNN','NKE','NJR','NRIM','NRZ','NTAP','O','OC','OKE','OZK','PEBO','PEP',
              'PFG','PG','PII','PLD','PM','PNC','PRU','RCL','RGLD','RNR','RF',
              'SBUX','SKT','SNV','SO','SPG','STAG','SWM','TGT','TM',
              'TSLA','TWO','TXN','UHT','UNH','UNM','UVV','V','VGR','VZ','WBA','WBS','WELL',
              'WEN','WFC','WM','WLK','WMT','XOM','YORW']

assets = pd.Series(portfolio + candidates).drop_duplicates().to_list()
print('\nCheck for duplicates!')
np.sort(candidates)


Check for duplicates!


array(['AAPL', 'ABBV', 'ABT', 'ACN', 'ADI', 'AGM', 'AGNC', 'ALB', 'AMTD',
       'AMZN', 'APD', 'ARR', 'AVAL', 'BABA', 'BAH', 'BEN', 'BLK', 'CAH',
       'CATY', 'CBT', 'CCBG', 'CCL', 'CFG', 'CHD', 'CHRW', 'CL', 'CMA',
       'CMCSA', 'CNI', 'COST', 'CSCO', 'CVS', 'CVX', 'D', 'DDS', 'DUK',
       'DX', 'EBF', 'EMR', 'ENB', 'EPD', 'EPR', 'ES', 'FAST', 'FB', 'FDX',
       'FITB', 'FLIC', 'FLO', 'FRT', 'GGG', 'GIS', 'GOOG', 'GS', 'GWB',
       'HBAN', 'IBCP', 'IBM', 'IMBBY', 'INTC', 'IRM', 'JNJ', 'JPM', 'K',
       'KEY', 'KMB', 'KO', 'LMT', 'LOW', 'MA', 'MAIN', 'MBWM', 'MCD',
       'MCY', 'MDP', 'MDT', 'MED', 'MMM', 'MMP', 'MNRL', 'MO', 'MPLX',
       'MSFT', 'MSI', 'MSM', 'NFLX', 'NHI', 'NJR', 'NKE', 'NNN', 'NRIM',
       'NRZ', 'NTAP', 'O', 'OC', 'OKE', 'OZK', 'PEBO', 'PEP', 'PFG', 'PG',
       'PII', 'PLD', 'PM', 'PNC', 'PRU', 'RCL', 'RF', 'RGLD', 'RNR',
       'SBUX', 'SKT', 'SNV', 'SO', 'SPG', 'STAG', 'SWM', 'TGT', 'TM',
       'TSLA', 'TWO', 'TXN', 'UHT', 'UNH', 'UNM', 'UVV', 'V',

In [6]:
data = yf.download(assets, period='10y').sort_index().resample('D').last()

prices = data['Adj Close'].fillna(method="ffill")
close = data['Close'].fillna(method="ffill")

(prices.count()[prices.count() < prices.count().quantile(0.05)]/365).sort_values()

[*********************100%***********************]  141 of 141 completed


MNRL    0.715068
CFG     5.282192
AVAL    5.284932
BABA    5.295890
NRZ     6.679452
ABBV    7.008219
MPLX    7.194521
dtype: float64

In [7]:
for a in prices.count()[prices.isna().sum() > 3*365].index:
    if a not in portfolio:
        prices = prices.drop(a, axis=1)
        print('Droping ' + a + ' for lack of historical prices.')

Droping AVAL for lack of historical prices.
Droping BABA for lack of historical prices.
Droping CFG for lack of historical prices.
Droping MNRL for lack of historical prices.
Droping NRZ for lack of historical prices.


In [34]:
def extractSignal(df1, df2, cols):
    try:
        df = df2[(df2[cols] == df1[cols]).sum(axis=1) == len(cols)].index
    except:
        print(df2[cols], df1[cols], cols)
    return df

def momentum(close, asset='Asset', step=50, output='current', chart=False, printOut=False):
    shortBuy = close > close.rolling(step).mean()
    mediumBuy = close > close.rolling(step * 2).mean()
    longBuy = close > close.rolling(step * 4).mean()

    medMBuy = close.rolling(step).mean() > close.rolling(step * 2).mean()
    longMBuy = close.rolling(step).mean() > close.rolling(step * 4).mean()
    veryLongBuy = close.rolling(step * 2).mean() > close.rolling(step * 4).mean()

    buySignals = pd.DataFrame(index=close.index)

    buySignals['BuySS'] = shortBuy
    buySignals['BuySM'] = mediumBuy
    buySignals['BuySL'] = longBuy
    buySignals['BuyMM'] = medMBuy
    buySignals['BuyML'] = longMBuy
    buySignals['BuyLL'] = veryLongBuy

    buySignals['BuySSc'] = 0
    buySignals['BuySMc'] = 0
    buySignals['BuySLc'] = 0
    buySignals['BuyMMc'] = 0
    buySignals['BuyMLc'] = 0
    buySignals['BuyLLc'] = 0

    buySignals['BuySSp'] = 0.0
    buySignals['BuySMp'] = 0.0
    buySignals['BuySLp'] = 0.0
    buySignals['BuyMMp'] = 0.0
    buySignals['BuyMLp'] = 0.0
    buySignals['BuyLLp'] = 0.0

    fwdRets = pd.Series(np.log(close/close.shift(1))).fillna(0.0)
    buySignals['RetS'] = fwdRets.rolling(int(step/2)).sum().shift(int(step/2))
    buySignals['RetM'] = fwdRets.rolling(step).sum().shift(step)
    buySignals['RetL'] = fwdRets.rolling(step * 4).sum().shift(step * 4)

    signalCols = ['BuySS','BuySM','BuySL','BuyMM','BuyML','BuyLL']
    signCountCols = ['BuySSc','BuySMc','BuySLc','BuyMMc','BuyMLc','BuyLLc']
    sCols = signalCols + signCountCols

    for cCol in signCountCols:
        buySignals[cCol] = pd.to_numeric(buySignals[cCol])

    for i in buySignals.index:
        for a in signalCols:
            if buySignals.loc[i, a]: 
                buySignals.loc[i, str(a) + 'c'] = buySignals.shift(1).loc[i, str(a) + 'c'] + 1
            else: 
                buySignals.loc[i, str(a) + 'c'] = 0
            
            bsInd = buySignals[buySignals[a] == buySignals.loc[i, a]].index
            rets = buySignals.loc[bsInd].loc[:i, 'RetM']
            buySignals.loc[i, str(a) + 'p'] = rets[rets >= 0.001].count()/rets.count()
    
    for cCol in signCountCols:
        buySignals[cCol][np.logical_and(buySignals[cCol] > 0, buySignals[cCol] <= 7)] = 1
        buySignals[cCol][np.logical_and(buySignals[cCol] > 7, buySignals[cCol] <= 14)] = 2
        buySignals[cCol][np.logical_and(buySignals[cCol] > 14, buySignals[cCol] <= 28)] = 3
        buySignals[cCol][np.logical_and(buySignals[cCol] > 28, buySignals[cCol] <= 90)] = 4
        buySignals[cCol][buySignals[cCol] > 28] = 5
    
    if output == 'current':
        return buySignals.iloc[-1]
    elif output == 'short':
        return buySignals.tail(step)
    elif output == 'full':
        return buySignals

for a in portfolio:
    print(a)
    print(momentum(prices[a], asset=a, step=30, output='current'))

T


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

BuySS          True
BuySM          True
BuySL          True
BuyMM          True
BuyML          True
BuyLL          True
BuySSc            3
BuySMc            3
BuySLc            5
BuyMMc            1
BuyMLc            5
BuyLLc            5
BuySSp     0.554811
BuySMp     0.640261
BuySLp     0.692931
BuyMMp     0.724074
BuyMLp      0.76425
BuyLLp     0.735533
RetS      0.0215509
RetM     -0.0205229
RetL       0.184313
Name: 2020-01-03 00:00:00, dtype: object
PFE
BuySS          True
BuySM          True
BuySL          True
BuyMM          True
BuyML          True
BuyLL          True
BuySSc            4
BuySMc            4
BuySLc            4
BuyMMc            4
BuyMLc            4
BuyLLc            4
BuySSp     0.600827
BuySMp      0.66215
BuySLp     0.706128
BuyMMp     0.737579
BuyMLp     0.749469
BuyLLp     0.738534
RetS      0.0215286
RetM      0.0125064
RetL      -0.101291
Name: 2020-01-03 00:00:00, dtype: object
FB
BuySS           True
BuySM           True
BuySL           True
BuyMM   

In [0]:
for a in portfolio:
    print(a)
    print(pd.DataFrame(momentumBuy(prices[a], a, 
                                   output='current'),
                       columns=list(['LAG','Signal','Profit Prob.','Return'])))

In [0]:
mret = prices.pct_change(28).fillna(0.0)
mret['Portfolio'] = mret[portfolio].mean(axis=1)

mlogret = np.log(prices/prices.shift(1)).rolling(28).sum().fillna(0.0)
mlogret['Portfolio'] = mlogret[portfolio].mean(axis=1)

mlogret['Portfolio'].describe()

In [0]:
profiles = pd.DataFrame(columns=list(['Name', 'Industry', 'Sub-Industry']), 
                        index=prices.columns, dtype=str)
profiles['Action'] = ''
profiles['Profit Prob.'] = 0.0
profiles['Beta'] = 0.0
profiles['RET'] = mret[profiles.index].mean().mul(12)
profiles['STD'] = mret[profiles.index].std().mul(np.sqrt(12))

i = 1
for a in prices.columns:
    try:
        profile_page = requests.get('https://finance.yahoo.com/quote/' + str(a) + '/profile?p=' + str(a))
        soup = BeautifulSoup(profile_page.content, 'html.parser')

        #print(a)

        profiles.loc[a, 'Name'] = soup.find('h3', {'class':"Fz(m) Mb(10px)"}).text

        for span in soup.find_all('span', {'class':'Fw(600)'}):
            if str(span).find('data-reactid="21"') > 0: 
                if len(span.text) > 3: profiles.loc[a, 'Industry'] = span.text
            elif str(span).find('data-reactid="22"') > 0:
                if len(span.text) > 3: profiles.loc[a, 'Industry'] = span.text
            elif str(span).find('data-reactid="23"') > 0:
                if len(span.text) > 3: profiles.loc[a, 'Industry'] = span.text
        
            if str(span).find('data-reactid="25"') > 0:
                if len(span.text)>3:profiles.loc[a, 'Sub-Industry'] = span.text
            elif str(span).find('data-reactid="26"') > 0:
                if len(span.text)>3:profiles.loc[a, 'Sub-Industry'] = span.text
            elif str(span).find('data-reactid="27"') > 0:
                if len(span.text)>3:profiles.loc[a, 'Sub-Industry'] = span.text
        
        aact = momentumBuy(prices[a], asset=a, output='single')
        curr = momentumBuy(prices[a], asset=a, output='current')
        
        profiles.loc[a, 'Action'] = aact
        profiles.loc[a, 'Profit Prob.'] = round(100*curr[1][2],2)
        
    except:
        print(a)
        profiles = profiles.drop(a)
        prices = prices.drop(a, axis=1)

    proc = round(i/len(prices.columns),3)
    #print(i, proc, proc % 0.1)
    if proc % 0.1 >= 0.095:
        print('{}% processed.'.format(100 * proc))
    i += 1

profiles.info()

In [0]:
profiles['Beta'] = mlogret.cov()['Portfolio']/mlogret.drop('Portfolio', axis=1).var().loc[profiles.index]

In [0]:
ret = prices.pct_change().dropna()
wret = prices.asfreq(freq='W', method='ffill').pct_change().dropna()
wretRoll = prices.pct_change(7).dropna()

log_ret = np.log(prices.div(prices.shift(1))).fillna(0.0)
alog_ret = log_ret.rolling(365).sum()

ret['Portfolio'] = ret[portfolio].mean(axis=1)
log_ret['Portfolio'] = log_ret[portfolio].mean(axis=1).fillna(0.0)

In [0]:
#Need to add Price to Earnings and Price to Book to profiles
profiles.loc[portfolio].sort_values(by=[ 'Action', 'Profit Prob.'], ascending=False)

In [0]:
profiles.loc[mlogret[mlogret['Portfolio'] < 0.0].corr()['Portfolio'].sort_values().head(15).index].sort_values(by='Profit Prob.')

In [0]:
portfolio_ind = profiles.loc[portfolio].groupby('Industry').count()['Name']
portfolio_subind = profiles.loc[portfolio].groupby('Sub-Industry').count()['Name']

print(portfolio_subind)
selection = profiles
industryCount = len(profiles.groupby('Industry').count().index)
subIndustryCount = len(profiles.groupby('Sub-Industry').count().index)

port_Ind_fully_invested = len(portfolio_ind.index) == industryCount
port_subInd_fully_invested = len(portfolio_subind.index) == subIndustryCount

for i in portfolio_ind.index:
    if port_Ind_fully_invested:
        if portfolio_ind.loc[i] == portfolio_ind.min():
            selection = selection[selection['Industry'] == i]
        else:
            selection = selection[selection['Industry'] != i]
    else:
        selection = selection[selection['Industry'] != i]
for i in portfolio_subind.index:
    if port_subInd_fully_invested:
        if portfolio_subind.loc[i] == portfolio_subind.min():
            selection = selection[selection['Industry'] == i]
        else:
            selection = selection[selection['Industry'] != i]
    else:
        selection = selection[selection['Sub-Industry'] != i]

selection = selection[selection['Action'] != 'SELL']

selection.sort_values(by=['Industry', 'Profit Prob.'])

In [0]:
profiles.loc[['NKE','BAH','SO','TGT','K']]

In [0]:
val = prices[selection.index].iloc[-1]/prices[selection.index].iloc[-365:].mean()
val = val[val < 1.0]
underVal = []

for a in val.index.to_list():
    if np.logical_and(prices[a].iloc[-3*365:].pct_change().mean() > 0.0,
                      a not in portfolio):
        underVal.append(a)

print('Currently assets under consideration:\n{}'.format(underVal))

In [0]:
if len(underVal) > 1:
    wretRoll[underVal].plot(subplots=True, figsize=(16,8))
    plt.show();
    wretRoll[underVal].mul(52).describe()

In [0]:
selAss = selection.index.to_list()
low_corr = ret.corr()[ret.corr()['Portfolio'] <= ret.corr()['Portfolio'].quantile(0.05)]

sns.heatmap(mlogret.corr().loc[selAss, selAss])
plt.show();

print('Lowest correlating assets:')
profiles.loc[low_corr.index]

In [0]:
sel = selection.index.to_list()[:]
sel.append('Portfolio')

sel_corr = ret[sel].corr()['Portfolio']
print(sel_corr.sort_values().head(10))

diversifiedPicks = sel_corr.sort_values()[sel_corr<sel_corr.quantile(0.25)].index.to_list()

In [0]:
profiles['LOGRET'] = log_ret.mean().mul(365)
profiles['LOGSTD'] = log_ret.std().mul(np.sqrt(365))

profiles.plot(kind='scatter', x='LOGSTD', y='LOGRET')#, size='Beta')
plt.show()

In [0]:
log_ret[sel].boxplot(figsize=(17,8))
plt.show()

candidatesSeries = pd.Series(index=sel)
riskSeries = pd.Series(index=sel)
for n in sel:
    t = portfolio[:]
    t.append(n)
    
    mu = log_ret[t].mean(axis=1).mean()*365
    cov = log_ret[t].cov()
    w = np.ones(len(t))
    w /= np.sum(w)
    risk = np.sqrt(365 * np.dot(w.T, np.dot(cov, w)))

    riskSeries.loc[n] = risk
    candidatesSeries.loc[n] = mu/risk
    print('{}: {}%, {}%, {}'.format(n, round(100*mu,2), 
                                    round(100*risk,2), 
                                    round(mu/risk, 2)))

bestSH = candidatesSeries[candidatesSeries > candidatesSeries['Portfolio']]
bestSH = bestSH[bestSH > bestSH.quantile(0.75)].index.to_list()

lowestRisk = riskSeries[riskSeries < riskSeries['Portfolio']]
lowestRisk = lowestRisk[lowestRisk < lowestRisk.quantile(0.25)].index.to_list()

print('\nBased on Return/Risk ratio, best pick would be {}'.format(bestSH))
print('Based on lowest portfolio Risk, best pick would be {}'.format(lowestRisk))

In [0]:
sns.heatmap(log_ret.corr(method='pearson').loc[selection.index, portfolio], annot=True)
plt.show()

bestPick = pd.Series(data=np.zeros(len(selection.index)), index=selection.index)

for b in bestPick.index:
    pear = pearsonr(log_ret['Portfolio'], log_ret[b])
    spear = spearmanr(log_ret['Portfolio'], log_ret[b])

    pSum = pear[1]+spear[1]
    if np.logical_and(pear[1]<0.05, spear[1]<0.05):
        bestPick.loc[b] = pear[0]*pear[1]/pSum + spear[0]*spear[1]/pSum
    elif pear[1]<0.05:
        bestPick.loc[b] = pear[0]
    elif spear[1]<0.05:
        bestPick.loc[b] = spear[0]

bestPick = bestPick.dropna()
bestCOR = bestPick[bestPick < bestPick.quantile(0.25)].sort_values().index.to_list()

print(bestPick.sort_values().head(10))
print('\nBased on correlations, best picks would be:')#\n{}'.format(bestCOR))

profiles.loc[bestCOR].sort_values(by='Profit Prob.', ascending=False)

In [0]:
n = 100
simRet = pd.DataFrame(columns=sel, index=list(range(n)))
simRisk = pd.DataFrame(columns=sel, index=list(range(n)))

for asset in sel:
    simRet[asset] = 0.0
    simRisk[asset] = 0.0

    for i in range(n):
        mu = ret[asset].mean()
        sigma = ret[asset].std()
        
        simulations = (np.ones(n*365) * mu -(sigma**2)/2) + np.random.standard_normal(n*365) * sigma
        
        simRet.loc[i, asset] = np.mean(simulations) * 365
        simRisk.loc[i, asset] = np.std(simulations) * np.sqrt(365)


bestRisk = simRisk.mode().median().sort_values()
bestRisk = bestRisk[bestRisk < bestRisk.quantile(0.25)]

bestReturns = simRet.mode().median()
bestReturns = bestReturns[bestReturns>bestReturns.quantile(0.75)]

bestSharpe = bestReturns/bestRisk
bestSharpe = bestSharpe[bestSharpe > bestSharpe.quantile(0.75)].sort_values(ascending=False)

if 'Portfolio' in bestSharpe.index: bestSharpe = bestSharpe.drop('Portfolio')
if 'Portfolio' in bestRisk.index: bestRisk = bestRisk.drop('Portfolio')
if 'Portfolio' in bestReturns.index: bestReturns = bestReturns.drop('Portfolio')

bestReturns.sort_values(ascending=False)

In [0]:
invetable = []

if len(bestSH)>=1:
    #print('\nBased on Return/Risk ratio, best picks would be:')
    for p in bestSH:
        #print(p)
        invetable.append(p)

if len(lowestRisk)>=1:
    #print('\nBased on Risk, best picks would be:')
    for p in lowestRisk:
        #print(p)
        invetable.append(p)

if len(bestCOR)>=1:
    #print('\nBased on correlations, best picks would be:')
    for p in bestCOR:
        #print(p)
        invetable.append(p)

if len(underVal)>=1:
    #print('\nBased on historical prices these stocks seam undervalued:')
    for p in underVal:
        #print(p)
        invetable.append(p)

try:
    if len(bestSharpe.index)>=1:
        #print('\nBased on simulated Return/Risk ratio, best picks would be:')        
        for p in bestSharpe.index.to_list():
            #print(p)
            invetable.append(p)

    if len(bestRisk.index):
        #print('\nBased on simulated Risk, best picks would be:')        
        for p in bestRisk.index.to_list():
            #print(p)
            invetable.append(p)

    if len(bestReturns.index)>=1:
        #print('\nBased on simulated Return, best picks would be:')        
        for p in bestReturns.index.to_list():
            #print(p)
            invetable.append(p)
except:
    print('\nNo simulations were made.')
    
invDF = pd.DataFrame(invetable).groupby(0)[0].count()
print('\nMost frequently picked assets:')
profiles.loc[invDF.sort_values(ascending=False)[invDF==invDF.max()].index]

In [0]:
selection[selection['Beta'] <= selection['Beta'].quantile(0.25)].sort_values(by='Profit Prob.',
                                                                             ascending=False)

In [0]:
profiles[np.logical_and(profiles['Beta'] <= profiles['Beta'].quantile(0.25), profiles['Action'] != 'SELL')].sort_values(by='Beta')

In [0]:
profiles[profiles['Action'] == 'BUY'].sort_values(by='Profit Prob.',
                                                  ascending=False)

In [0]:
profiles[profiles['Profit Prob.'] > 85.0].sort_values(by='Profit Prob.', ascending=False)